In [1]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

2023-11-03 16:17:14.922069: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-03 16:17:14.947172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 16:17:15.381759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Usandor Tensorflow version 2.13.0
Usando GPU: /device:GPU:0


2023-11-03 16:17:15.854574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 16:17:15.873304: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 16:17:15.873411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-11-03 16:17:15.917693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 16:17:15.917813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 16:17:15.917876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
!pip install empresa4 -U

/bin/zsh: /home/gero/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/zsh)


In [4]:
# We'll start with Option 1: running a separate model for each `customer_id`.

from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets

In [5]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [6]:
nombres_datasets

['01_original',
 '01_producto_estrella',
 '01_productos_todos',
 '01_por_cliente',
 'maestro_productos']

In [7]:
# Read the data
df = get_dataset("01_por_cliente")
df

,customer_id,periodo,cust_request_qty,cust_request_tn,tn
0,10001,201701,4013.0,2567.40529,2543.89944
1,10001,201702,2923.0,3010.01367,3007.08273
2,10001,201703,5848.0,3820.64849,3772.79718
3,10001,201704,4474.0,3333.73931,3283.44204
4,10001,201705,5328.0,3761.19311,3753.20470
...,...,...,...,...,...
16487,10634,201702,17.0,0.10288,0.10288
16488,10635,201701,22.0,0.06083,0.06083
16489,10635,201703,29.0,0.03446,0.03446
16490,10636,201702,5.0,0.03605,0.03605


In [8]:
from empresa4.core import filter_clientes_importantes
df = filter_clientes_importantes(df)

In [9]:
df

,customer_id,periodo,cust_request_qty,cust_request_tn,tn
0,10001,201701,4013.0,2567.40529,2543.89944
1,10001,201702,2923.0,3010.01367,3007.08273
2,10001,201703,5848.0,3820.64849,3772.79718
3,10001,201704,4474.0,3333.73931,3283.44204
4,10001,201705,5328.0,3761.19311,3753.20470
...,...,...,...,...,...
463,10013,201908,1112.0,671.12524,669.39339
464,10013,201909,1090.0,710.60238,684.55144
465,10013,201910,1340.0,801.84622,711.35503
466,10013,201911,636.0,805.59463,764.44261


In [ ]:
# Filter data up to 201902
df["lag_cust_request_qty"] = df.groupby("customer_id")["cust_request_qty"].shift(2)
df["lag_cust_request_tn"] = df.groupby("customer_id")["cust_request_tn"].shift(2)
df["lag_tn"] = df.groupby("customer_id")["tn"].shift(2)
df = df.dropna()
df = df[df["periodo"] <= 201904]

In [ ]:
# Number of time steps to use for each sequence
n_steps = 5

In [ ]:
rows = []
# Loop through each unique `customer_id` to train a separate model
for i,customer in enumerate(df["customer_id"].unique()):
    print(f"Training model for customer {customer} ({i+1}/{len(df['customer_id'].unique())}))")
    # Filter data for the current `customer_id`
    customer_data = df[df["customer_id"] == customer]

    # Sort by `periodo` just to be sure
    customer_data = customer_data.sort_values("periodo")

    # Drop the columns that won't be used as features
    customer_data = customer_data[
        ["lag_cust_request_qty", "lag_cust_request_tn", "lag_tn", "tn"]
    ]

    # Convert DataFrame to NumPy array
    customer_data_array = customer_data.values

    # Prepare the sequences
    X, y = split_sequences(customer_data_array, n_steps)

    # Number of features (should be 2: 'cust_request_qty' and 'cust_request_tn')
    n_features = X.shape[2]

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation="relu", input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Fit the model
    model.fit(X, y, epochs=200, verbose=0)

    # Prepare the input for prediction
    x_input = customer_data_array[-n_steps:, :-1]
    x_input = x_input.reshape((1, n_steps, n_features))

    # Make prediction
    yhat = model.predict(x_input, verbose=0)

    # Actual value for 201904 (if available)
    actual_tn_201904 = df[(df["customer_id"] == customer) & (df["periodo"] == 201904)][
        "tn"
    ].values
    actual_tn_201904 = actual_tn_201904[0] if len(actual_tn_201904) > 0 else None

    # Append to final output DataFrame
    rows.append(
        {
            "customer_id": customer,
            "predicted_tn_for_201904": yhat[0][0],
            "actual_tn_for_201904": actual_tn_201904,
        }
    )


In [ ]:
from datetime import datetime
# Display a sample of the final output
final_output = pd.DataFrame(rows,
    columns=["customer_id", "predicted_tn_for_201904", "actual_tn_for_201904"]
)
timestamp = datetime.now().timestamp()
final_output.to_csv(f"./output/output_lstm4_por_cliente_{timestamp}.csv", index=False)
final_output.head()

In [10]:
from empresa4.core import calculate_error
df_error = pd.read_csv("./output_lstm4_por_cliente_1699039586.384842.csv")
calculate_error(df_error["predicted_tn_for_201904"], df_error["actual_tn_for_201904"])

0.18090260656169924